# 5次式の2次式と3次式の積への因数分解の仕方

* 黒木玄

In [1]:
using SymPy

# Override the Base.show definition of SymPy.jl:
# https://github.com/JuliaPy/SymPy.jl/blob/29c5bfd1d10ac53014fa7fef468bc8deccadc2fc/src/types.jl#L87-L105

@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::SymbolicObject)
    print(io, as_markdown("\\displaystyle " * sympy.latex(x, mode="plain", fold_short_frac=false)))
end
@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::AbstractArray{Sym})
    function toeqnarray(x::Vector{Sym})
        a = join(["\\displaystyle " * sympy.latex(x[i]) for i in 1:length(x)], "\\\\")
        """\\left[ \\begin{array}{r}$a\\end{array} \\right]"""
    end
    function toeqnarray(x::AbstractArray{Sym,2})
        sz = size(x)
        a = join([join("\\displaystyle " .* map(sympy.latex, x[i,:]), "&") for i in 1:sz[1]], "\\\\")
        "\\left[ \\begin{array}{" * repeat("r",sz[2]) * "}" * a * "\\end{array}\\right]"
    end
    print(io, as_markdown(toeqnarray(x)))
end

## 一般論

In [2]:
@vars a b c d e p q r s t u v x

(a, b, c, d, e, p, q, r, s, t, u, v, x)

$t\ne 0$ と仮定する.

In [3]:
f = x^5 + p*x^4 + q*x^3 + r*x^2 + s*x + t

   4      3      2              5
p*x  + q*x  + r*x  + s*x + t + x 

In [4]:
g = x^2 + a*x + b

           2
a*x + b + x 

In [5]:
h = x^3 + c*x^2 + d*x + e

   2              3
c*x  + d*x + e + x 

$f$ を $g$ で割ったときの商と余りを求める.

In [6]:
quotient, remainder = sympy.div(f, g, x)
sympy.poly.([quotient, remainder], x)

2-element Vector{Sym}:
                                            Poly(x^3 + (-a + p)*x^2 + (a^2 - a*p - b + q)*x - a^3 + a^2*p + 2*a*b - a*q - b*p + r, x, domain='ZZ[p,q,r,a,b]')
 Poly((a^4 - a^3*p - 3*a^2*b + a^2*q + 2*a*b*p - a*r + b^2 - b*q + s)*x + a^3*b - a^2*b*p - 2*a*b^2 + a*b*q + b^2*p - b*r + t, x, domain='ZZ[p,q,r,s,t,a,b]')

$h$ が商に一致することと $c,d,e$ が次のようになることは同値.

In [7]:
C, D, E = [quotient.coeff(x, k) for k in 2:-1:0]

3-element Vector{Sym}:
                               -a + p
                    a^2 - a*p - b + q
 -a^3 + a^2*p + 2*a*b - a*q - b*p + r

$be = t\ne 0$ ならば $b\ne 0$ でなければいけない.

そのとき, 余りが0になることと以下の2つが0になることは同値である.

In [8]:
eqrem0 = remainder.coeff(x, 0)/b |> expand

 3    2                             t
a  - a *p - 2*a*b + a*q + b*p - r + -
                                    b

In [9]:
eqrem1 = remainder.coeff(x, 1)

 4    3        2      2                      2          
a  - a *p - 3*a *b + a *q + 2*a*b*p - a*r + b  - b*q + s

与えられた5次でモニックな整数係数多項式 $f$ の整数係数の2次式 $g$ と3次式 $h$ の積への分解を求めるためには, まず $be=t$ を満たす整数の組 $(b, e)$ の各々について, `eqrem0` を0にする整数 $a$ が存在するかどうかを確認し, 存在する場合に `eqrem1` が0になるかどうかも確認し, `eqrem0` と `eqrem1` の両方を $0$ にするような整数の組 $(a,b)$ を見付ければよい.  (見付からない場合には整数係数の2次式と整数係数の3次式の積に分解不可能なことが証明されたことになる.)

## 雑多な計算メモ

In [10]:
sympy.div(f, x^3 + a*x^2 + b*x + c, x) |> collect

2-element Vector{Sym}:
                                                                                             a^2 - a*p - b + q + x^2 + x*(-a + p)
 -a^2*c + a*c*p + b*c - c*q + t + x^2*(-a^3 + a^2*p + 2*a*b - a*q - b*p - c + r) + x*(-a^2*b + a*b*p + a*c + b^2 - b*q - c*p + s)

In [11]:
sympy.div(x^6 + p*x^5 + q*x^4 + r*x^3 + s*x^2 + t*x + u, g, x) |> collect

2-element Vector{Sym}:
                                 a^4 - a^3*p - 3*a^2*b + a^2*q + 2*a*b*p - a*r + b^2 - b*q + s + x^4 + x^3*(-a + p) + x^2*(a^2 - a*p - b + q) + x*(-a^3 + a^2*p + 2*a*b - a*q - b*p + r)
 -a^4*b + a^3*b*p + 3*a^2*b^2 - a^2*b*q - 2*a*b^2*p + a*b*r - b^3 + b^2*q - b*s + u + x*(-a^5 + a^4*p + 4*a^3*b - a^3*q - 3*a^2*b*p + a^2*r - 3*a*b^2 + 2*a*b*q - a*s + b^2*p - b*r + t)

In [12]:
Q63, R63 = sympy.div(x^6 + p*x^5 + q*x^4 + r*x^3 + s*x^2 + t*x + u, x^3 + a*x^2 + b*x + c, x) |> collect

2-element Vector{Sym}:
                                                                                                                                                 -a^3 + a^2*p + 2*a*b - a*q - b*p - c + r + x^3 + x^2*(-a + p) + x*(a^2 - a*p - b + q)
 a^3*c - a^2*c*p - 2*a*b*c + a*c*q + b*c*p + c^2 - c*r + u + x^2*(a^4 - a^3*p - 3*a^2*b + a^2*q + 2*a*b*p + 2*a*c - a*r + b^2 - b*q - c*p + s) + x*(a^3*b - a^2*b*p - a^2*c - 2*a*b^2 + a*b*q + a*c*p + b^2*p + 2*b*c - b*r - c*q + t)

In [13]:
C63 = [R63.coeff(x, k) for k in 2:-1:0]

3-element Vector{Sym}:
       a^4 - a^3*p - 3*a^2*b + a^2*q + 2*a*b*p + 2*a*c - a*r + b^2 - b*q - c*p + s
 a^3*b - a^2*b*p - a^2*c - 2*a*b^2 + a*b*q + a*c*p + b^2*p + 2*b*c - b*r - c*q + t
                         a^3*c - a^2*c*p - 2*a*b*c + a*c*q + b*c*p + c^2 - c*r + u

In [14]:
B63 = sympy.solve(C63[end], b)[end]

 3      2                2          
a *c - a *c*p + a*c*q + c  - c*r + u
------------------------------------
            c*(2*a - p)             

In [15]:
@show eqrem63_1 = -(C63[1](b=>B63)(u=>c*v) * (-2*a + p)^2).expand().factor();

eqrem63_1 = -(((((C63[1])(b => B63))(u => c * v) * (-2a + p) ^ 2).expand()).factor()) = a^6 - 3*a^5*p + 3*a^4*p^2 + 2*a^4*q - 4*a^3*c - a^3*p^3 - 4*a^3*p*q + 4*a^3*v + 7*a^2*c*p + 2*a^2*p^2*q + a^2*p*r - 5*a^2*p*v + a^2*q^2 - 4*a^2*s - 4*a*c*p^2 - a*p^2*r + 2*a*p^2*v - a*p*q^2 + 4*a*p*s - c^2 + c*p^3 - c*p*q + 2*c*r - 2*c*v - p^2*s + p*q*r - p*q*v - r^2 + 2*r*v - v^2


In [16]:
[eqrem63_1.coeff(a, k) for k in 6:-1:0]

7-element Vector{Sym}:
                                                                                1
                                                                             -3*p
                                                                      3*p^2 + 2*q
                                                         -4*c - p^3 - 4*p*q + 4*v
                                        7*c*p + 2*p^2*q + p*r - 5*p*v + q^2 - 4*s
                                       -4*c*p^2 - p^2*r + 2*p^2*v - p*q^2 + 4*p*s
 -c^2 + c*p^3 - c*p*q + 2*c*r - 2*c*v - p^2*s + p*q*r - p*q*v - r^2 + 2*r*v - v^2

In [17]:
sympy.poly(eqrem63_1, a).coeffs()

7-element Vector{Sym}:
                                                                                1
                                                                             -3*p
                                                                      3*p^2 + 2*q
                                                         -4*c - p^3 - 4*p*q + 4*v
                                        7*c*p + 2*p^2*q + p*r - 5*p*v + q^2 - 4*s
                                       -4*c*p^2 - p^2*r + 2*p^2*v - p*q^2 + 4*p*s
 -c^2 + c*p^3 - c*p*q + 2*c*r - 2*c*v - p^2*s + p*q*r - p*q*v - r^2 + 2*r*v - v^2

## 例1

$g = x^2 + 3x - 2$, $h = (x+1)^3-2 = x^3+3x^2+3x-1$ の場合にどうなるか.

In [18]:
GG = x^2 + 3x - 2

 2          
x  + 3*x - 2

In [19]:
HH = (x+1)^3-2 |> expand

 3      2          
x  + 3*x  + 3*x - 1

In [20]:
FF = GG*HH

/ 2          \ / 3      2          \
\x  + 3*x - 2/*\x  + 3*x  + 3*x - 1/

In [21]:
FF = expand(FF)

 5      4       3      2          
x  + 6*x  + 10*x  + 2*x  - 9*x + 2

既存の函数を使えば因数分解は易しい.

In [22]:
factor(FF)

/ 2          \ / 3      2          \
\x  + 3*x - 2/*\x  + 3*x  + 3*x - 1/

係数を $P,Q,R,S,T$ に代入する.

In [23]:
_, P, Q, R, S, T = [FF.coeff(x, k) for k in 5:-1:0]

6-element Vector{Sym}:
  1
  6
 10
  2
 -9
  2

余りの定数項が0になるという条件.

In [24]:
Eqrem0 = eqrem0(p=>P, q=>Q, r=>R, s=>S, t=>T)

 3      2                            2
a  - 6*a  - 2*a*b + 10*a + 6*b - 2 + -
                                     b

余りの1次の項が0になるという条件.

In [25]:
Eqrem1 = eqrem1(p=>P, q=>Q, r=>R, s=>S, t=>T)

 4      3      2         2                   2           
a  - 6*a  - 3*a *b + 10*a  + 12*a*b - 2*a + b  - 10*b - 9

$be = 2$ となる整数の組 $(b, e)$ 全体について, 最下段の方程式が整数解を持つ場合を探す.

In [26]:
[Eqrem0(b=>k) |> factor for k in (1, 2, -1, -2)]

4-element Vector{Sym}:
   a^3 - 6*a^2 + 8*a + 6
  a^3 - 6*a^2 + 6*a + 11
 a^3 - 6*a^2 + 12*a - 10
 (a - 3)*(a^2 - 3*a + 5)

最後の $b=-2$ の場合にのみ整数解 $a=3$ が存在することがわかった.

In [27]:
BB, AA = -2, 3

(-2, 3)

この場合が実際に解になっていることの確認.

In [28]:
C, D, E

(-a + p, a^2 - a*p - b + q, -a^3 + a^2*p + 2*a*b - a*q - b*p + r)

In [29]:
CC, DD, EE = C(a=>AA, b=>BB, p=>P), D(a=>AA, b=>BB, p=>P, q=>Q), E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T)

(3, 3, -1)

In [30]:
Eqrem1(a=>AA, b=>BB)

0

In [31]:
sol = (x^2 + AA*x + BB)*(x^3 + CC*x^2 + DD*x + EE)

/ 2          \ / 3      2          \
\x  + 3*x - 2/*\x  + 3*x  + 3*x - 1/

In [32]:
sol - FF |> simplify

0

## 例2

$f = x^5 - x^4 - 1$ の因数分解.

In [33]:
FF = x^5 - x^4 - 1

 5    4    
x  - x  - 1

既存の函数を使えば易しい.

In [34]:
factor(FF)

/ 2        \ / 3        \
\x  - x + 1/*\x  - x - 1/

よくある解答例では天下り的に「$x^3$ を足して引く」という方法を使っている.

$$
\begin{aligned}
x^5 - x^4 - 1
&=
x^5 - x^4 + x^3 - x^3 - 1
\\ &=
(x^5 - x^4 + x^3) - (x^3 + 1)
\\ &=
x^3(x^2-x+1) -(x+1)(x^2-x+1)
\\ &=
(x^2-x+1)(x^3-x-1).
\end{aligned}
$$

以下の方法であればそういう天下り的な操作が無用になる.

係数を $P,Q,R,S,T$ に代入する.

In [35]:
_, P, Q, R, S, T = [FF.coeff(x, k) for k in 5:-1:0]

6-element Vector{Sym}:
  1
 -1
  0
  0
  0
 -1

余りの定数項が0になるという条件.

In [36]:
Eqrem0 = eqrem0(p=>P, q=>Q, r=>R, s=>S, t=>T)

 3    2               1
a  + a  - 2*a*b - b - -
                      b

余りの1次の項が0になるという条件.

In [37]:
Eqrem1 = eqrem1(p=>P, q=>Q, r=>R, s=>S, t=>T)

 4    3      2              2
a  + a  - 3*a *b - 2*a*b + b 

$be=-1$ となる整数の組 $(b,d)$ の各々について, 最下段の方程式が整数解を持つかどうかを確認する.

In [38]:
[(B = k; E = T/k; Eqrem0(b=>B, e=>E) |> factor) for k in (1, -1)]

2-element Vector{Sym}:
 (a + 1)*(a^2 - 2)
 (a + 1)*(a^2 + 2)

$b=\pm 1$ の両方の場合に整数解 $a=-1$ を持つことがわかった.

これらが下から2段目の方程式も満たしているかを確認する.

In [39]:
BB, AA = 1, -1
Eqrem1(a=>AA, b=>BB)

0

In [40]:
BB, AA = -1, -1
Eqrem1(a=>AA, b=>BB)

2

$(a, b) = -1, 1$ の場合のみが解になっていることがわかった.

以下は検算である.

In [41]:
AA, BB = -1, 1
CC, DD, EE = C(a=>AA, b=>BB, p=>P), D(a=>AA, b=>BB, p=>P, q=>Q), E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T)

(0, -1, -1)

In [42]:
sol = (x^2 + AA*x + BB)*(x^3 + CC*x^2 + DD*x + EE)

/ 2        \ / 3        \
\x  - x + 1/*\x  - x - 1/

In [43]:
sol - FF |> simplify

0

## 例3

$f = x^5 + 16x + 32$ の因数分解.

In [44]:
FF = x^5 + 16x + 32

 5            
x  + 16*x + 32

既存の函数を使えば易しい.

In [45]:
factor(FF)

/ 2          \ / 3      2    \
\x  + 2*x + 4/*\x  - 2*x  + 8/

係数を $P,Q,R,S,T$ に代入する.

In [46]:
_, P, Q, R, S, T = [FF.coeff(x, k) for k in 5:-1:0]

6-element Vector{Sym}:
  1
  0
  0
  0
 16
 32

余りの定数項が0になるという条件.

In [47]:
Eqrem0 = eqrem0(p=>P, q=>Q, r=>R, s=>S, t=>T)

 3           32
a  - 2*a*b + --
             b 

余りの1次の項が0になるという条件.

In [48]:
Eqrem1 = eqrem1(p=>P, q=>Q, r=>R, s=>S, t=>T)

 4      2      2     
a  - 3*a *b + b  + 16

$b$ として $32$ の約数全体を動かして, `eqrem0` を0にする整数 $a$ が存在する場合を探す.

In [49]:
[[k, Eqrem0(b=>k) |> factor] for k in ((2 .^ (0:5))..., (-2 .^ (0:5))...)] |> (x -> stack(x; dims=1))

12×2 Matrix{Sym}:
   1           a^3 - 2*a + 32
   2           a^3 - 4*a + 16
   4  (a - 2)*(a^2 + 2*a - 4)
   8           a^3 - 16*a + 4
  16           a^3 - 32*a + 2
  32           a^3 - 64*a + 1
  -1           a^3 + 2*a - 32
  -2  (a - 2)*(a^2 + 2*a + 8)
  -4            a^3 + 8*a - 8
  -8           a^3 + 16*a - 4
 -16           a^3 + 32*a - 2
 -32           a^3 + 64*a - 1

方程式 `eqrem0 = 0` は $b=4,-2$ のときにのみ整数解 $a=2$ を持つことがわかった.

In [50]:
BB, AA = 4, 2
Eqrem1(a=>AA, b=>BB)

0

In [51]:
BB, AA = -2, 2
Eqrem1(a=>AA, b=>BB)

60

方程式 `eqrem0 = eqrem1 = 0` の整数解 $a = 2$, $b=4$ が見付かった.

In [52]:
AA, BB = 2, 4
CC, DD, EE = C(a=>AA, b=>BB, p=>P), D(a=>AA, b=>BB, p=>P, q=>Q), E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T)

(-2, 0, 8)

In [53]:
sol = (x^2 + AA*x + BB)*(x^3 + CC*x^2 + DD*x + EE)

/ 2          \ / 3      2    \
\x  + 2*x + 4/*\x  - 2*x  + 8/

In [54]:
sol - FF |> simplify

0

## 例4

$f = x^5 + 3x^4 - 2x^3 - 2x^2 - 6x + 4$ の因数分解

In [55]:
FF = x^5 + 3x^4 - 2x^3 - 2x^2 - 6x + 4

 5      4      3      2          
x  + 3*x  - 2*x  - 2*x  - 6*x + 4

In [56]:
factor(FF)

/ 3    \ / 2          \
\x  - 2/*\x  + 3*x - 2/

$f$ が $x^k + a$ 型の因子を持つと決め打ちできるなら, そのように $f$ を整理すれば容易に因数分解できる.

$$
\begin{aligned}
x^5 + 3x^4 - 2x^3 - 2x^2 - 6x + 4
&=
(x^5 + 3x^4 - 2x^3) - (2x^2 + 6x - 4)
\\ &=
x^3(x^2 + 3x - 2) - 2(x^2 + 3x - 2)
\\ &=
(x^3-2)(x^2 + 3x - 2).
\end{aligned}
$$

In [57]:
factor(FF)

/ 3    \ / 2          \
\x  - 2/*\x  + 3*x - 2/

In [58]:
_, P, Q, R, S, T = [FF.coeff(x, k) for k in 5:-1:0]

6-element Vector{Sym}:
  1
  3
 -2
 -2
 -6
  4

In [59]:
eqrem0

 3    2                             t
a  - a *p - 2*a*b + a*q + b*p - r + -
                                    b

In [60]:
Eqrem0 = eqrem0(p=>P, q=>Q, r=>R, s=>S, t=>T)

 3      2                           4
a  - 3*a  - 2*a*b - 2*a + 3*b + 2 + -
                                    b

In [61]:
eqrem1

 4    3        2      2                      2          
a  - a *p - 3*a *b + a *q + 2*a*b*p - a*r + b  - b*q + s

In [62]:
Eqrem1 = eqrem1(p=>P, q=>Q, r=>R, s=>S, t=>T)

 4      3      2        2                  2          
a  - 3*a  - 3*a *b - 2*a  + 6*a*b + 2*a + b  + 2*b - 6

In [63]:
[Eqrem0(b=>k, e=>T/k) |> factor for k in (1, 2, 4, -1, -2, -4)]

6-element Vector{Sym}:
   a^3 - 3*a^2 - 4*a + 9
  a^3 - 3*a^2 - 6*a + 10
 a^3 - 3*a^2 - 10*a + 15
         a^3 - 3*a^2 - 5
       (a - 3)*(a^2 + 2)
  a^3 - 3*a^2 + 6*a - 11

In [64]:
BB, AA = -2, 3
Eqrem1(a=>AA, b=>BB)

0

In [65]:
AA, BB = 3, -2
CC, DD, EE = C(a=>AA, b=>BB, p=>P), D(a=>AA, b=>BB, p=>P, q=>Q), E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T)

(0, 0, -2)

In [66]:
sol = (x^2 + AA*x + BB)*(x^3 + CC*x^2 + DD*x + EE)

/ 3    \ / 2          \
\x  - 2/*\x  + 3*x - 2/

In [67]:
sol - FF |> simplify

0